In [1]:
import numpy as np 
import pandas as pd 
import os, cv2
from tqdm import tqdm
from sklearn.model_selection import StratifiedShuffleSplit

from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.applications import *

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
x_train = []
y_train = []

label_map = {   "Black-grass"               :0,
                "Charlock"                  :1,
                "Cleavers"                  :2,
                "Common Chickweed"          :3,
                "Common wheat"              :4,
                "Fat Hen"                   :5,
                "Loose Silky-bent"          :6,
                "Maize"                     :7,
                "Scentless Mayweed"         :8,
                "Shepherds Purse"           :9,
                "Small-flowered Cranesbill" :10,
                "Sugar beet"                :11}

dim = 299

# Preparing training data
dirs = os.listdir("/input/plant_seedlings_classification/train/")
for k in tqdm(range(len(dirs))):    # Directory
    files = os.listdir("/input/plant_seedlings_classification/train/{}".format(dirs[k]))
    for f in range(len(files)):     # Files
        img = cv2.imread('/input/plant_seedlings_classification/train/{}/{}'.format(dirs[k], files[f]))
        targets = np.zeros(12)
        targets[label_map[dirs[k]]] = 1 
        x_train.append(cv2.resize(img, (dim, dim)))
        y_train.append(targets)
    
y_train = np.array(y_train, np.uint8)
x_train = np.array(x_train, np.float32)

print(x_train.shape)
print(y_train.shape)

#x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.01, random_state=42)
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.16, random_state=42) # Want a balanced split for all the classes
for train_index, test_index in sss.split(x_train, y_train):
    print("Using {} for training and {} for validation".format(len(train_index), len(test_index)))
    x_train, x_valid = x_train[train_index], x_train[test_index]
    y_train, y_valid = y_train[train_index], y_train[test_index]

100%|██████████| 12/12 [02:01<00:00,  8.88s/it]


(4750, 299, 299, 3)
(4750, 12)


In [5]:
datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range = 40,
        horizontal_flip=True)
                                  
val_datagen = ImageDataGenerator(rescale = 1./255)
                                      
weights = os.path.join('', '/output/weights.h5')

epochs = 100
learning_rate = 0.001
batch_size = 32

callbacks = [ EarlyStopping(monitor='val_loss', patience=5, verbose=0), 
              ModelCheckpoint(weights, monitor='val_loss', save_best_only=True, verbose=0),
              ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=0, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0)]

base_model = InceptionV3(input_shape=(dim, dim, 3), include_top=False, weights='imagenet', pooling='avg') # Average pooling reduces output dimensions
x = base_model.output
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(12, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Training FCN on top
for layer in base_model.layers:
    layer.trainable = False
    
model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=learning_rate), metrics=['accuracy'])

85565440/87910968 [============================>.] - ETA: 0s

In [8]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=len(x_train)/batch_size, 
                    validation_data=val_datagen.flow(x_valid, y_valid, batch_size=batch_size), 
                    validation_steps=len(x_valid)/batch_size,
                    callbacks=callbacks,
                    epochs=30, 
                    verbose=1)

Epoch 1/30
125/124 [==============================] - 233s - loss: 1.1383 - acc: 0.6498 - val_loss: 6.7509 - val_acc: 0.3237
Epoch 2/30
125/124 [==============================] - 207s - loss: 0.6516 - acc: 0.8020 - val_loss: 2.0757 - val_acc: 0.5382
Epoch 3/30
125/124 [==============================] - 207s - loss: 0.4453 - acc: 0.8476 - val_loss: 0.7841 - val_acc: 0.7487
Epoch 4/30
125/124 [==============================] - 206s - loss: 0.4157 - acc: 0.8683 - val_loss: 8.1770 - val_acc: 0.2737
Epoch 5/30
125/124 [==============================] - 206s - loss: 0.3836 - acc: 0.8762 - val_loss: 2.0852 - val_acc: 0.6434
Epoch 6/30
125/124 [==============================] - 207s - loss: 0.3076 - acc: 0.8912 - val_loss: 3.0793 - val_acc: 0.6303
Epoch 7/30
125/124 [==============================] - 207s - loss: 0.1847 - acc: 0.9442 - val_loss: 0.1500 - val_acc: 0.9553
Epoch 8/30
125/124 [==============================] - 207s - loss: 0.1404 - acc: 0.9539 - val_loss: 0.1044 - val_acc: 0.9684


In [18]:
# fine tuning 

model.load_weights(weights)

for layer in model.layers[:172]:
    layer.trainable = False
for layer in model.layers[172:]:
    layer.trainable = True
    
finetune_weights = os.path.join('', '/output/finetune_weights.h5')

learning_rate = 0.0001
batch_size = 32

callbacks = [ EarlyStopping(monitor='val_loss', patience=5, verbose=0), 
              ModelCheckpoint(finetune_weights, monitor='val_loss', save_best_only=True, verbose=0),
              ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0)]

model.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=learning_rate), metrics=['accuracy'])


In [19]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=len(x_train)/batch_size, 
                    validation_data=val_datagen.flow(x_valid, y_valid, batch_size=batch_size), 
                    validation_steps=len(x_valid)/batch_size,
                    callbacks=callbacks,
                    epochs=50, 
                    verbose=1)

Epoch 1/50
125/124 [==============================] - 116s - loss: 0.0775 - acc: 0.9775 - val_loss: 0.0967 - val_acc: 0.9724
Epoch 2/50
125/124 [==============================] - 111s - loss: 0.0887 - acc: 0.9721 - val_loss: 0.1051 - val_acc: 0.9750
Epoch 3/50
125/124 [==============================] - 112s - loss: 0.0713 - acc: 0.9760 - val_loss: 0.0814 - val_acc: 0.9724
Epoch 4/50
125/124 [==============================] - 111s - loss: 0.0754 - acc: 0.9762 - val_loss: 0.1253 - val_acc: 0.9632
Epoch 5/50
125/124 [==============================] - 111s - loss: 0.0769 - acc: 0.9754 - val_loss: 0.1062 - val_acc: 0.9697
Epoch 6/50
124/124 [============================>.] - ETA: 0s - loss: 0.0737 - acc: 0.9783Epoch 00005: val_loss did not improve

Epoch 00005: reducing learning rate to 9.999999747378752e-06.
125/124 [==============================] - 113s - loss: 0.0796 - acc: 0.9767 - val_loss: 0.1049 - val_acc: 0.9671
Epoch 7/50
125/124 [==============================] - 111s - loss: 0.0

In [20]:
# ------ TESTING ------
x_test = []
df_test = pd.read_csv('/input/plant_seedlings_classification/sample_submission.csv')

for f, species in tqdm(df_test.values, miniters=100):
    img = cv2.imread('/input/plant_seedlings_classification/test/{}'.format(f))
    x_test.append(cv2.resize(img, (dim, dim)))

x_test = np.array(x_test, np.float32)
print(x_test.shape)

x_test = x_test /255.0

if os.path.isfile(finetune_weights):
    model.load_weights(finetune_weights)

p_test = model.predict(x_test, verbose = 1)

preds = []
for i in range(len(p_test)):
    pos = np.argmax(p_test[i])
    preds.append(list(label_map.keys())[list(label_map.values()).index(pos)])

df_test['species'] = preds
df_test.to_csv('/output/submission.csv', index=False)

100%|██████████| 794/794 [00:05<00:00, 143.06it/s]


(794, 299, 299, 3)
